<a href="https://colab.research.google.com/github/garthajon/QuantFinanceIntro/blob/main/LIQUIDITY-MEANREVERSION-HEALTHYUPWARD-SCREENER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

import time
import requests
import pandas as pd
import yfinance as yf
from tqdm import tqdm

# -----------------------------
# Step 1: Scrape S&P 500 tickers from Wikipedia
# -----------------------------
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}
response = requests.get(wiki_url, headers=headers)
tables = pd.read_html(response.text)
sp500_df = tables[0]
tickers = sp500_df["Symbol"].tolist()
print(f"Loaded {len(tickers)} tickers")

# -----------------------------
# Screening parameters
# -----------------------------
lookback_years = 2
rolling_window = 12  # months
min_data_months = rolling_window + 2  # need at least 2 prior months

# Container for recovery screen
recovery_healthy = []

# -----------------------------
# Helper function: compute monthly z-scores
# -----------------------------
def compute_monthly_zscores(ticker):
    try:
        df = yf.download(ticker, period=f"{lookback_years}y", auto_adjust=True, progress=False)
        if df.empty:
            return None
    except Exception:
        return None

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    # Daily spread
    df["spread_ret"] = (df["High"] - df["Low"]) / df["Close"].shift(1)

    # Monthly aggregation
    monthly = pd.DataFrame({
        "close": df["Close"].resample("ME").last(),
        "volume": df["Volume"].resample("ME").mean(),
        "spread": df["spread_ret"].resample("ME").mean()
    }).dropna()

    if monthly.shape[0] < min_data_months:
        return None

    # Returns
    monthly["returns"] = monthly["close"].pct_change()

    # Rolling stats
    r_ret = monthly["returns"].shift(1)
    r_vol = monthly["volume"].shift(1)
    r_spr = monthly["spread"].shift(1)

    monthly["ret_z"] = (monthly["returns"] - r_ret.rolling(rolling_window).mean()) / r_ret.rolling(rolling_window).std()
    monthly["vol_z"] = (monthly["volume"] - r_vol.rolling(rolling_window).mean()) / r_vol.rolling(rolling_window).std()
    monthly["spr_z"] = (monthly["spread"] - r_spr.rolling(rolling_window).mean()) / r_spr.rolling(rolling_window).std()

    monthly = monthly.dropna()
    if monthly.shape[0] < rolling_window:
        return None

    return monthly

# -----------------------------
# Step 2: Loop through tickers and apply recovery healthy filter
# -----------------------------
for ticker in tqdm(tickers):
    monthly = compute_monthly_zscores(ticker)
    time.sleep(0.5)

    if monthly is None or monthly.shape[0] < 2:
        continue

    latest = monthly.iloc[-1]
    prior = monthly.iloc[-2]  # previous month

    # Conditions
    ret_z_current = latest["ret_z"]
    ret_z_prior   = prior["ret_z"]
    vol_z_current = latest["vol_z"]
    spr_z_current = latest["spr_z"]

    # Recovery Healthy Trend Filter
    # the current z score is an improvement upward on the the prior return zscore
    # and the the zscore for the volume is currently strong and the spread is currently steady in terms of its zscore
    if (-1.2 <= ret_z_current <= 1.5) and (ret_z_prior <= -0.5) and (vol_z_current >= 0.5) and (spr_z_current <= 0.5):
        recovery_healthy.append(ticker)

# -----------------------------
# Step 3: Output
# -----------------------------
print("\nStocks meeting Recovery Healthy Trend Scenario:")
print(recovery_healthy)



/tmp/ipython-input-108363341.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


Loaded 503 tickers


 12%|█▏        | 61/503 [00:42<05:16,  1.40it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')
 15%|█▌        | 76/503 [00:52<04:57,  1.43it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=2y)')
100%|██████████| 503/503 [05:47<00:00,  1.45it/s]


Stocks meeting Recovery Healthy Trend Scenario:
['BA', 'CARR', 'CPRT', 'EQIX', 'FFIV', 'HD', 'HPQ', 'KMB', 'LYV', 'MSFT', 'MOS', 'MSI', 'MSCI', 'NCLH', 'PYPL', 'SNPS', 'TEL', 'URI', 'VICI', 'V', 'DIS', 'ZTS']
